In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sys
from datetime import timedelta  

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    
    
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [2]:
data_et = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et.csv')
data_et_fix = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_fix.csv')
data_et_choice = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_choice.csv')

data_trial = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial.csv')
data_trial_fix = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_fix.csv')
data_trial_choice = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_choice.csv')

data_subject = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_subject.csv') \
    .rename(columns=
            {
                'chosenAmount': 'bonus_pay',
                'chosenDelay': 'bonus_delay'
            }
           )    

overview = pd.DataFrame(
    [
        [len(data_et)],
        [len(data_et_fix)],
        [len(data_et_choice)],
        [len(data_trial)],
        [len(data_trial_fix)],
        [len(data_trial_choice)],
        [len(data_subject)]
    ], 
    columns=['length'],
    index=['data_et', 'data_et_fix', 'data_et_choice', 
           'data_trial', 'data_trial_fix', 'data_trial_choice', 
           'data_subject'])
print(overview)

                   length
data_et            618566
data_et_fix         57762
data_et_choice     148345
data_trial          30362
data_trial_fix        918
data_trial_choice    3655
data_subject           84


# Approve subjects

In [3]:
if 'trial_length' in data_subject.columns: 
    print('trial_length already added!')
else:
    output = []
    for subject in data_trial['run_id'].unique():
        prolificID = data_trial.loc[data_trial['run_id']==subject, 'prolificID'].unique()[0]
        trial_length = len(data_trial.loc[data_trial['run_id']==subject, 'trial_index'].unique())
        output.append([subject, prolificID, trial_length])
    output = pd.DataFrame(output,
                 columns=['run_id', 'prolificID', 'trial_length']
                )

    data_subject = data_subject.merge(output, on=['run_id', 'prolificID'], how='left')

In [4]:
data_et = data_et.merge(
    data_trial.loc[:, ['run_id', 'chinFirst', 'trial_index', 'trial_type', 'task_nr']], 
    on=['run_id', 'trial_index'], 
    how='left'
)

In [5]:
output = []
for subject in data_et['run_id'].unique():
    m_x_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].mean()
    m_x_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].mean()

    m_y_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'y'
    ].mean()
    m_y_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'y'
    ].mean()

    m_count_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].count()

    m_count_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].count()
    
    m_choseTop = data_trial_choice.loc[data_trial_choice['run_id']==subject,
                                       'choseTop'].mean()

    output.append([subject, m_x_fix, m_y_fix, m_x_choice, m_y_choice, 
                   m_count_fix, m_count_choice, 
                   m_choseTop])

output = pd.DataFrame(output, 
                      columns=['run_id', 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 
                               'm_count_fix', 'm_count_choice',
                               'm_choseTop'])
output

data_subject = data_subject.merge(output, on='run_id', how='left')

## These subjects await review

In [6]:
data_subject.loc[data_subject['status']=='AWAITING REVIEW', 
                 ['run_id', 'prolificID', 'trial_length',
                 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
                 'm_choseTop']
                ].sort_values(by='prolificID')

,run_id,prolificID,trial_length,m_x_fix,m_y_fix,m_x_choice,m_y_choice,m_count_fix,m_count_choice,m_choseTop


In [7]:
if 'choice_rt' in data_subject.columns:
    print('choice_rt already added!')
else: 
    grouped = data_trial.loc[data_trial['trial_type']=='eyetracking-choice', :] \
        .groupby(['prolificID'])['trial_duration_exact'].mean() \
        .reset_index() \
        .rename(columns={'trial_duration_exact': 'choice_rt'})
    data_subject = data_subject.merge(grouped, on='prolificID', how='left')

data_subject.loc[
    data_subject['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'webcam_fps', 'choice_rt', 'bonus_pay', 'bonus_delay']
] \
    .sort_values(by='run_id')

,run_id,prolificID,webcam_fps,choice_rt,bonus_pay,bonus_delay


# Basic payment

In [8]:
data_pay = data_subject.loc[data_subject['status']=='APPROVED', 
                            [
                                'run_id', 'prolificID', 
                                'age',  'Country of Birth', 'Current Country of Residence', 'First Language',
                                'Nationality', 'Sex',
                                'status', 'reviewed_at_datetime', 'Country of Birth', 'entered_code',
                                'session_id', 'started_datetime', 'completed_date_time', 'time_taken',
                                'bonus_pay', 'bonus_delay'
                            ]
                           ]

data_pay['basic_pay'] = 2.25
data_pay.loc[:, ['bonus_pay', 'bonus_delay']]

,bonus_pay,bonus_delay
1,$4.5,180 days
4,$3.5,Today
5,NaN,NaN
8,$4.5,90 days
9,$4.5,7 days
10,$5,30 days
11,$4,Today
12,$3.5,Today
14,$4.5,7 days
15,$3,Today


# Bonus payment

Reformat payments

In [24]:
def choice_options_to_numeric(data, varName): 
    data[varName] = data[varName].astype(str)
    data.loc[: , [varName]] = \
        data.loc[:, [varName]] \
            .replace(['Today', 'Tomorrow', '7 days', 
                      '15 days', '30 days', '90 days', 
                      '180 days'], 
                     [0, 1, 7, 15, 30, 90, 180]) \
            .replace({'\$':''}, regex = True) \
            .replace('50 cent', 0.5) \
            .astype(float)
    return data

for var in ['bonus_pay', 'bonus_delay']:
    data_pay = choice_options_to_numeric(data_pay, var)

In [26]:
data_pay[['run_id', 'prolificID', 'bonus_pay', 'bonus_delay']]

,run_id,prolificID,bonus_pay,bonus_delay
0,103,600063f2943eab0acc812ed8,4.5,180.0
1,11,5c5684ef9d244c0001b29f1e,3.5,0.0
2,12,5b8969006651ea000118e42e,5.0,1.0
3,126,5c0e4e3d87e876000151cfec,4.5,90.0
4,128,5f01a0893963d02ea1d65d12,4.5,7.0
5,13,5b4e65de09f5af0001080f45,5.0,30.0
6,130,5fea19869bf07d97b4a87ae8,4.0,0.0
7,131,5e53a7b4d9a7ea3a2b062da0,3.5,0.0
8,19,5f860a189bc695131f22ec9c,4.5,7.0
9,24,5f77a902fc647a327b77004a,3.0,0.0


## Replace empty values with the best bonus reward

In [27]:
# For two subjects, the bonus payment was not recorded
data_pay.loc[
    data_pay['prolificID'].isin(
        ['5fea6632bf9ae4a79153efdf', 
         '5b8969006651ea000118e42e', 
         '5fb2af792942a58ffe303948', 
        ]
    ), ['bonus_pay', 'bonus_delay']] = [5, 1]

data_pay.loc[data_pay[['bonus_pay', 'bonus_delay']].isnull().any(axis=1), :]

,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,...,session_id,started_datetime,completed_date_time,time_taken,bonus_pay,bonus_delay,basic_pay,date,completed_date,reward_pound


## What bonus is due today? 

In [28]:
def add_completed_date(data, data_trial):
    output = []

    for subject in data_trial['run_id'].unique():
        thisSubject = data_trial.loc[data_trial['run_id']==subject] \
            .reset_index(drop=True)
        date_time_obj = datetime.datetime.strptime(
            thisSubject.loc[0, 'recorded_at'], '%Y-%m-%d %H:%M:%S')

        output.append([thisSubject.loc[0, 'run_id'], date_time_obj.date()])
        
    output = pd.DataFrame(output, columns=['run_id', 'date']) 
    
    if 'date' in data.columns: data = data.drop(columns=['date'])
    data = data.merge(output, on='run_id', how='left')
    return data

data_pay = add_completed_date(data_pay, data_trial)
    
data_pay['completed_date']=data_pay.apply(
    lambda x: datetime.datetime.strptime(
        x['completed_date_time'], '%Y-%m-%d %H:%M:%S.%f') \
            .date(),
    axis=1)
data_pay['bonus_pay'].unique()

array([4.5, 3.5, 5. , 4. , 3. , 2.5, 0.5, 2. ])

In [29]:
data_pay['reward_pound'] = data_pay['bonus_pay'] * 0.75

In [31]:
data_pay['due_on'] = data_pay['completed_date'] +  data_pay['bonus_delay'].map(datetime.timedelta) 

data_pay.loc[:, 
               [
                   'prolificID', 'run_id',
                   'Nationality', 'Sex',  
                   'completed_date', 'bonus_pay', 'bonus_delay', 
                   'reward_pound'
               ]
              ]. sort_values(by='completed_date')

,prolificID,run_id,Nationality,Sex,completed_date,bonus_pay,bonus_delay,reward_pound
24,5ea3f75481a44e39aa318256,5,Israel,Male,2021-01-18,4.5,30.0,3.375
28,5f4e08aafb74230a29cb4ef0,6,Poland,Male,2021-01-18,5.0,15.0,3.750
40,5fea6632bf9ae4a79153efdf,9,Serbia,Male,2021-01-18,5.0,1.0,3.750
20,5f946436ae32a127565d9405,43,Portugal,Male,2021-01-18,4.0,30.0,3.000
19,5fcbb3066829ff1a601fe080,42,United Kingdom,Female,2021-01-18,5.0,1.0,3.750
18,5f881c7d29650a26096e77a3,41,United Kingdom,Male,2021-01-18,4.5,30.0,3.375
17,5f96fc2552ce3f040af63bbc,4,Spain,Male,2021-01-18,4.5,30.0,3.375
16,5f1e9db721e4f101783fe21c,38,Poland,Male,2021-01-18,4.0,1.0,3.000
14,5f8472685956c40c720d0936,36,Italy,Male,2021-01-18,0.5,0.0,0.375
13,5ea94df3a879f002f3e9a812,32,Poland,Female,2021-01-18,4.5,90.0,3.375


In [32]:
bonus_due_today = data_pay.loc[data_pay['due_on']==datetime.datetime.now().date(), :]
bonus_due_today['reward_pound'] = bonus_due_today['reward_pound'].round(2)
bonus_due_today

<ipython-input-32-e0c5874ff982>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonus_due_today['reward_pound'] = bonus_due_today['reward_pound'].round(2)


,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,...,started_datetime,completed_date_time,time_taken,bonus_pay,bonus_delay,basic_pay,completed_date,reward_pound,date,due_on
4,128,5f01a0893963d02ea1d65d12,26.0,Germany,Greece,Greek,Greece,Male,APPROVED,2021-01-19 12:36:47.163000,...,2021-01-19 10:23:21.887000,2021-01-19 11:01:30.874000,2288.987,4.5,7.0,2.25,2021-01-19,3.38,2021-01-19,2021-01-26
23,48,5fa1192cf99e161a5cfad1cd,18.0,Poland,Poland,DATA EXPIRED,Poland,Male,APPROVED,2021-01-19 11:00:41.549000,...,2021-01-19 09:09:18.617000,2021-01-19 09:44:13.747000,2095.130,4.0,7.0,2.25,2021-01-19,3.00,2021-01-19,2021-01-26
34,70,5d430fdf871f1700017b2a81,49.0,Spain,Spain,Catalan,Spain,Male,APPROVED,2021-01-19 10:41:24.134000,...,2021-01-19 09:10:03.087000,2021-01-19 10:41:23.434000,5480.347,5.0,7.0,2.25,2021-01-19,3.75,2021-01-19,2021-01-26
37,80,5fa69c24315bfb2d0a37c685,20.0,Poland,Poland,DATA EXPIRED,Poland,Male,APPROVED,2021-01-19 11:00:49.360000,...,2021-01-19 09:09:12.814000,2021-01-19 09:44:37.014000,2124.200,4.0,7.0,2.25,2021-01-19,3.00,2021-01-19,2021-01-26


In [33]:
bonus_due_today.loc[: , ['prolificID', 'reward_pound']] \
    .to_csv('C:/Users/User/Google Drive/Masterarbeit/Acquise/bonusToday.csv', 
            index=False, 
            header=False
           )
print(bonus_due_today['due_on'].unique())

[datetime.date(2021, 1, 26)]


# Total costs

In [34]:
data_pay['basic_cost'] = data_pay['basic_pay'] * 1.4
data_pay['total_cost'] = (data_pay['basic_pay'] + data_pay['bonus_pay']) * 1.4
print('Basic Costs: ' + str(sum(data_pay['basic_cost'])))
print('Total Costs: ' + str(sum(data_pay['total_cost'])))
data_pay

Basic Costs: 154.35000000000014
Total Costs: 432.9499999999997


,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,...,time_taken,bonus_pay,bonus_delay,basic_pay,completed_date,reward_pound,date,due_on,basic_cost,total_cost
0,103,600063f2943eab0acc812ed8,19.0,Italy,Italy,NaN,Italy,Male,APPROVED,2021-01-19 10:58:01.081000,...,1760.968,4.5,180.0,2.25,2021-01-19,3.375,2021-01-19,2021-07-18,3.15,9.45
1,11,5c5684ef9d244c0001b29f1e,29.0,Hong Kong,United States,Chinese,United States,Male,APPROVED,2021-01-18 10:53:28.738000,...,1227.532,3.5,0.0,2.25,2021-01-18,2.625,2021-01-18,2021-01-18,3.15,8.05
2,12,5b8969006651ea000118e42e,22.0,Italy,Italy,Italian,Italy,Female,APPROVED,2021-01-18 10:42:25.881000,...,1403.632,5.0,1.0,2.25,2021-01-18,3.750,2021-01-18,2021-01-19,3.15,10.15
3,126,5c0e4e3d87e876000151cfec,29.0,Poland,Poland,Polish,Poland,Female,APPROVED,2021-01-19 12:36:10.712000,...,1912.745,4.5,90.0,2.25,2021-01-19,3.375,2021-01-19,2021-04-19,3.15,9.45
4,128,5f01a0893963d02ea1d65d12,26.0,Germany,Greece,Greek,Greece,Male,APPROVED,2021-01-19 12:36:47.163000,...,2288.987,4.5,7.0,2.25,2021-01-19,3.375,2021-01-19,2021-01-26,3.15,9.45
5,13,5b4e65de09f5af0001080f45,29.0,Poland,Poland,Polish,Poland,Male,APPROVED,2021-01-18 10:56:07.011000,...,1692.194,5.0,30.0,2.25,2021-01-18,3.750,2021-01-18,2021-02-17,3.15,10.15
6,130,5fea19869bf07d97b4a87ae8,19.0,Spain,Spain,NaN,Spain,Male,APPROVED,2021-01-19 13:24:23.588000,...,1349.099,4.0,0.0,2.25,2021-01-19,3.000,2021-01-19,2021-01-19,3.15,8.75
7,131,5e53a7b4d9a7ea3a2b062da0,40.0,United Kingdom,United Kingdom,English,United Kingdom,Female,APPROVED,2021-01-19 13:24:22.246000,...,1415.591,3.5,0.0,2.25,2021-01-19,2.625,2021-01-19,2021-01-19,3.15,8.05
8,19,5f860a189bc695131f22ec9c,20.0,Poland,Poland,DATA EXPIRED,Poland,Male,APPROVED,2021-01-18 10:57:59.489000,...,2116.522,4.5,7.0,2.25,2021-01-18,3.375,2021-01-18,2021-01-25,3.15,9.45
9,24,5f77a902fc647a327b77004a,27.0,United Kingdom,United Kingdom,DATA EXPIRED,United Kingdom,Female,APPROVED,2021-01-18 17:54:59.440000,...,2115.673,3.0,0.0,2.25,2021-01-18,2.250,2021-01-18,2021-01-18,3.15,7.35


# Export modified prolific data

In [35]:
data_pay.to_csv('C:/Users/User/Google Drive/Masterarbeit/Acquise/data_pay.csv', index=False, header=True)

# Feedback

In [36]:
print('Success! Script ran through.')

Success! Script ran through.
